In [89]:
import pandas as pd

Loading dataset

In [90]:
df = pd.read_csv('kindle_reviews_db.csv')
df.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [91]:
df = df[['reviewText', 'rating']]
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [92]:
df.shape

(12000, 2)

Missing values

In [93]:
df.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [94]:
df['rating'].unique()

array([3, 5, 4, 2, 1], dtype=int64)

In [95]:
## check balanced or imbalanced dfset
df['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

## 1. Data preprocessing and cleaning

In [96]:
# pos review = 1, neg review = 0
df['rating'] = df['rating'].apply(lambda x : 0 if x < 3 else 1)
df['rating']

0        1
1        1
2        1
3        1
4        1
        ..
11995    1
11996    1
11997    1
11998    0
11999    1
Name: rating, Length: 12000, dtype: int64

In [97]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


In [98]:
df['rating'].unique()

array([1, 0], dtype=int64)

In [99]:
df['rating'].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

#1 Lowercase all

In [100]:
df['reviewText'] = df['reviewText'].str.lower()
df['reviewText']

0        jace rankin may be short, but he's nothing to ...
1        great short read.  i didn't want to put it dow...
2        i'll start by saying this is the first of four...
3        aggie is angela lansbury who carries pocketboo...
4        i did not expect this type of book to be in li...
                               ...                        
11995    valentine cupid is a vampire- jena and ian ano...
11996    i have read all seven books in this series. ap...
11997    this book really just wasn't my cuppa.  the si...
11998    tried to use it to charge my kindle, it didn't...
11999    taking instruction is a look into the often hi...
Name: reviewText, Length: 12000, dtype: object

In [101]:
df.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [102]:
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

# Removing special characters
df['reviewText'] = df['reviewText'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', str(x)))
# Remove stopwords
stop_words = set(stopwords.words('english'))
df['reviewText'] = df['reviewText'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))
# Remove URLs
df['reviewText'] = df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://[\w\-_]+(\.[\w\-_]+)+([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '', str(x)))
# Remove HTML tags
df['reviewText'] = df['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())
# Remove additional spaces
df['reviewText'] = df['reviewText'].apply(lambda x: " ".join(x.split()))

In [103]:
df.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


Lemmatization

In [104]:
from nltk.stem import WordNetLemmatizer

In [105]:
lemmatizer = WordNetLemmatizer()

In [106]:
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [107]:
df['reviewText'] = df['reviewText'].apply(lambda x:lemmatize_words(x))

In [108]:
df.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


## 2. Train-Test Split

In [109]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['reviewText'], df['rating'], test_size=0.20)

## 3. Encoding sentences into vectors
- Bow, tf-Idf, word2vec

In [110]:
##Bow 
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer()
X_train_bow = bow.fit_transform(X_train).toarray()
X_test_bow = bow.transform(X_test).toarray()

In [111]:
##Bow 
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()

In [112]:
X_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

# 4. Training ML algorithm

In [113]:
from sklearn.naive_bayes import GaussianNB
nb_model_bow = GaussianNB().fit(X_train_bow, y_train)
nb_model_tfidf = GaussianNB().fit(X_train_tfidf, y_train)

In [114]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [115]:
y_pred_bow = nb_model_bow.predict(X_test_bow)
y_pred_tfidf = nb_model_tfidf.predict(X_test_tfidf)


In [116]:
confusion_matrix(y_test,y_pred_bow)

array([[531, 295],
       [693, 881]], dtype=int64)

In [117]:
print("BOW accuracy:", accuracy_score(y_test, y_pred_bow))

BOW accuracy: 0.5883333333333334


In [118]:
print("TfIdf accuracy:", accuracy_score(y_test, y_pred_tfidf))

TfIdf accuracy: 0.58875
